# 第12章 モノイド

- newtypeキーワード



In [1]:
newtype ZipList a = ZipList { getZipList :: [a]}

`data` が出ているのは7章

In [5]:
data X = X { name :: String} deriving (Show)

In [6]:
X {name = "hoge"}

X {name = "hoge"}

In [7]:
type Y = [X]

In [8]:
:t [X{name = "hoge"}, X{name="fuga"}]

[X{name = "hoge"}, X{name="fuga"}] :: [X]

In [13]:
data ZipList a = ZipList [a]

In [14]:
data ZipList a = ZipList { getZiplist :: [a]}

Haskellのnewtypeキーワードは　「1つの型をとり、それを何かにくるんで別の方に見せかけたい」という場合のために作られたもの


In [15]:
newtype ZipList a = ZipList { getZiplist :: [a]}


newtypeとdataを使った時の違いは？

- 高速
  - dataキーワードの場合オーバーヘッドがかかる
  - newtypeの場合既存の型をくるんで作った新しい方なので、内部処理は同じまま違う方をもたせている

じゃあ常にnewtypeを使わない理由は？

- newtypeキーワードを使って既存の方から新しい方を作る時は値コンストラクタは1種類しか作れない
- その値コンストラクタがもてるフィールドも1つだけ

In [16]:
newtype CharList = CharList { getCharList :: [Char]} deriving (Eq, Show)

Line 1: Use String
Found:
[Char]
Why not:
String

In [18]:
CharList "this will be shown!"
-- getCharList :: CharList -> [Char] という型を持つことになる

CharList {getCharList = "this will be shown!"}

### newtypeを使って型クラスのインスタンスを作る

ある型を型クラスのインスタンスにしたいが、型引数が一致しなくて出来ないということがあるらしい（わからない  
MaybeをFunctorインスタンスにするのは簡単  

Functor型クラスの定義  

```
class Functor f where
    fmap :: (a -> b) -> f a -> f b
```

なので、Maybe型コンストラクタがFunctor型クラスの定義の内fの位置を占めてしまい、全ての型引数が埋まる  
↓はMaybeに限定して動作する場合の書き方  

```
instance Functor Maybe where
    fmap :: (a -> b) -> Maybe a -> Maybe b
```

じゃあタプルにfmapを作用させたい場合は？  
こういう時は難しい  
Maybeをインスタンス化楽にできたのは型変数が１つの型コンストラクタだけがFunctorのインスタンスになれるから  
fmapが変更するのは型aの部分というのは指定できなさそう  
そこで `newtype` ですよ  

```
newtype Pair b a = Pair {getPair :: (a, b)}

instance Functor (Pair c) where
    fmap f (Pair (x, y)) = Pair(f x, y)
```


In [19]:
newtype Pair b a = Pair {getPair :: (a, b)}

instance Functor (Pair c) where
    fmap f (Pair (x, y)) = Pair(f x, y)

In [20]:
fmap (+1) (1,2)

(1,3)

In [21]:
getPair $ fmap (*100) (Pair (2,3))

(200,3)

### newtypeと遅延評価

newtypeで作られた型と元の型を型としては区別しつつ、同一の内部表現で扱っている  
パターンマッチがより怠惰になることを意味する(?)  


In [23]:
head [3,4,5, undefined]
-- undefinedが評価されなければ例外は投げられない

3

In [24]:
undefined

Prelude.undefined

In [27]:
data CoolBool = CoolBool {getCoolBool :: Bool}


In [28]:
helloMe :: CoolBool -> String
helloMe (CoolBool _) = "hello"

In [29]:
helloMe undefined

Prelude.undefined

In [30]:
newtype CoolBool = CoolBool { getCoolBool :: Bool}

In [32]:
helloMe :: CoolBool -> String
helloMe (CoolBool _) = "hello"

In [33]:
helloMe undefined

"hello"

dataキーワードで定義された型には複数の値コンストラクタがあるかもしれず、  
引数(CoolBool \_) に合致するかどうかを確認するために  
どのコンストラクタが使われたのかわかるところまで引数の評価を進める必要があるから `data` の場合例外になる  

しかし、newtypeキーワードはコンストラクタを1つしか作れないため、引数を評価すること無く初めのパターンマッチにマッチすることがわかる

### type vs newtype vs data

- type
  - 型シノニムを作るためのもの
  - 既存の型に別名を付けて、呼びやすくする
- newtype
  - 既存の型を包んで新しい型を作る
  - 型クラスのインスタンスを作りやすくするために使われる
  - 元の方とは別物になる
    - CharList ++ [Char] は出来ない
  - 新しい型は元の方の所属していた型クラスの引き継ぎは無いから色々やる必要はある
- data
  - 新しいものを作るときに使う
  

次回

# Monoid大集合

デュエルスタンバイ

* に対する 1 と ++ に対する [] は共通の性質を持っている  

- 関数は引数を２つ取る
- 2つの引数及び返り値の方は全て等しい
- 2引数関数を施して相手を変えないような特殊な値が存在する

* と ++ 結合的 (関数を値の間に挟む順序を変えても結果は変わらない)  

### Monoid型クラス

モノイドは結合的な二項演算子とその演算に関する単位元からなる構造体  
1は * の単位元  
[] は ++ の単位元  

```
class Monoid m where
    mempty :: m
    mappend :: m -> m -> m
    mconcat :: [m] -> m
    mconcat = foldr mappend mempty
```

Data.Monoidモジュールに定義されてる  
Monoidのインスタンスになれるのは具体型だけ  
mappendと言っているがappendというニュアンスではなく、2つのモノイド値を受け取って第3の値を返す関数  

### モノイド則

```
mempty `mapped` x = x
x `mappend` mempty = x
(x `mappend` y) `mappend` z = x `mappend` (y `mappend` z)
```

### モノイドとの遭遇

- リスト

```
instance Monoid [a] where
    mempty = []
    mappend = (++)
```



In [9]:
import Data.Monoid
[1,2,3] `mappend` [4,5,6]

[1,2,3,4,5,6]

### Maybeモノイド

Maybe aをモノイドにする方法1 - aがモノイドの時に限りMaybe　a もモノイドであるとする  

```
instance Monoid a => Monoid (Maybe a) where
    mempty = Nothing
    Nothing `mappend` m = m
    m `mappend` Nothing = m
    Just m1 `mappend Just m2 = Just (m1 `mappend` m2)
```

第一引数を返して、第二引数は捨てる  
First a  

```
newtype First a = First { getFirst :: Maybe a} deriving (Eq, Ord, Read, Show)

instance Monoid (First a) where
    mempty = First Nothing
    First (Just x) `mappend` _ = First (Just x)
    First Nothing `mappend` x = x
```

mappendは第二引数を無視している  

### モノイドで畳み込む

Foldable型クラス  


In [11]:
import qualified Data.Foldable as F

In [12]:
:t foldr

foldr :: forall (t :: * -> *) a b. Foldable t => (a -> b -> b) -> b -> t a -> b

In [14]:
:t F.foldr
-- どっちもFoldableになってる。本来Preludeはリストを受け取る仕様だったっぽい

F.foldr :: forall (t :: * -> *) a b. Foldable t => (a -> b -> b) -> b -> t a -> b

In [15]:
data Tree a = EmptyTree|Node a (Tree a) (Tree a) deriving (Show)

In [16]:
instance F.Foldable Tree where
    foldMap f EmptyTree = mempty
    foldMap f (Node x l r) = F.foldMap f l `mappend` f x `mappend` F.foldMap f r

In [17]:
t = Node 5 (Node 3 (Node 1 EmptyTree EmptyTree) (Node 6 EmptyTree EmptyTree)) (Node 9 (Node 8 EmptyTree EmptyTree) (Node 10 EmptyTree EmptyTree))

In [18]:
F.foldl (+) 0 t

42

In [19]:
F.foldl (*) 1 t

64800

In [20]:
foldl (+) 0 t

Line 1: Use sum
Found:
foldl (+) 0
Why not:
sum

42